<a href="https://colab.research.google.com/github/ammar0211/Mini-Projects/blob/master/REVISED_Data_Analyst_Internship_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# REVIEW SENTIMENT ANALYSIS

INSTALLING LIBRARIES

In [1]:
!pip install selenium #installing selenium for all reviews fetching
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver #installing chrome driver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin #copying driver to current directory 

     |████████████████████████████████| 911kB 4.6MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://secu

IMPORTING LIBRARIES

In [0]:
#Libraries to be used for Data Extraction
import requests,re
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import lxml
from lxml import html,etree
import time
from bs4 import BeautifulSoup

In [0]:
#Libraries used for Data Analysis
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from collections import Counter

In [4]:
nltk.download('stopwords') 
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [0]:
import spacy

In [0]:
#adding chrome driver to system path
import os,sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') 

In [8]:
#Configuring Chrome WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

#Importing the webdriver
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


ALLIED FUNCTIONS

In [0]:
#StopWords Preperation
rem={"aren't","below","didn't","didn'","don't","doesn't","doesn'","hadn't","hasn't","haven't","isn't","not","no","wasn't"}
stop_words=set(stopwords.words('english'))
stop_words-=rem

In [0]:
nlp=spacy.load("en", disable=['ner','parser','tagger'])
stopwords_lemma=set([word.lemma_ for word in nlp(' '.join(stop_words))]) #Lemmatizing stopwords

In [0]:
def clean_text_2(text):
  text=' '.join([word.lemma_ for word in nlp(text)]) #lemmatization
  tokenizer = RegexpTokenizer(r'\w+') #RegularExpression Tokenizer for removing any non-word characters
  text=' '.join([w for w in tokenizer.tokenize(text) if not w in stop_words]) #stopwords removal
  return text

In [0]:
#Cleaning Review Paragraph Text
def clean_text(df,col):
  df.iloc[:,col]=df.iloc[:,col].apply(lambda x: x.lower().strip()) #changing to lowercase & stripping whitespaces
  df.iloc[:,col]=df.iloc[:,col].apply(lambda x: re.sub('[^a-zA-Z0-9]',' ',x)) #removing characters other than a-z,A-Z,0-9
  df.iloc[:,col]=df.iloc[:,col].apply(lambda x: clean_text_2(x)) #lemmatization,removing stopwords and punctuations, keeping only ASCII word characters

In [0]:
# function to check if the button is on the page, to avoid miss-click problem
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [0]:
def nltk_sent_cal(text):
  sid = SentimentIntensityAnalyzer()
  return sid.polarity_scores(text)

DATA EXTRACTION FUNCTION

In [0]:
#Functin for Extracting all the restaurant reviews from the TripAdvisor website by using URL and providing Maximum Pages to look
def get_review(url,max_page=None):
  reviewList=[]
  driver.get(url) #Opening URL in webdriver
  soup=BeautifulSoup(driver.page_source,'lxml') #Extracting Source Code
  page=max_page if max_page else max(map(int,html.fromstring(str(soup)).xpath('//div[contains(@class,"ui_pagination")]//a[contains(@class,"pageNum")]//text()'))) #Finding maximum number of pages
  for _ in range(page):
      #expanding reviews for each page
      if (check_exists_by_xpath("//span[@class='taLnk ulBlueLinks']")):
        driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()
        time.sleep(3)
      soup=BeautifulSoup(driver.page_source,'lxml') #extraction of sourcecode of webpage
      review_lists=html.fromstring(str(soup)).xpath('.//div[@id="REVIEWS"]//div[contains(@class,"review-container")]') #Xpath for review container
      for review in review_lists:
        #Xpaths to each of the required data
        XPATH_REVIEW_ID='.//div[contains(@class,"reviewSelector")]/@data-reviewid'
        XPATH_RATING='.//div[contains(@class,"reviewSelector")]//span[contains(@class,"ui_bubble_rating")]/@class'
        XPATH_REVIEW_TITLE='.//div[contains(@class,"reviewSelector")]//span[@class="noQuotes"]//text()'
        XPATH_REVIEW_DATE='.//div[contains(@class,"reviewSelector")]//div[@data-prwidget-name="reviews_stay_date_hsx"]/text()'
        XPATH_PARAGRAPH='.//div[contains(@class,"reviewSelector")]//div[@data-prwidget-name="reviews_text_summary_hsx"]//p[@class="partial_entry"]//text()'
        
        #extracting data
        raw_review_id=review.xpath(XPATH_REVIEW_ID)
        raw_rating=review.xpath(XPATH_RATING)
        raw_review_title=review.xpath(XPATH_REVIEW_TITLE)
        raw_review_date=review.xpath(XPATH_REVIEW_DATE)
        raw_review_paragraph=review.xpath(XPATH_PARAGRAPH)
        
        #cleaning raw data
        ID=raw_review_id[0] if raw_review_id else None
        rating=list(raw_rating[0])[-2] if raw_rating else None
        title=raw_review_title[0] if raw_review_title else None
        date=raw_review_date[0] if raw_review_date else None
        paragraph=str(raw_review_paragraph[0]).replace('\n',' ') if raw_review_paragraph else None
        
        #storing data
        data = {
            'Site':url,
            #'ID': ID,
            'Rating':rating,
            'Review Title':title,
            'Review Date':date,
            'Review Paragraph':paragraph,
        }
        reviewList.append(data)
      
      #Moving to Next Page
      if (check_exists_by_xpath('*//div[contains(@class,"ui_pagination")]//a[@class="//span[@class="nav next ui_button primary disabled"]')):
        break
      if (check_exists_by_xpath('*//div[contains(@class,"ui_pagination")]//a[@class="nav next taLnk ui_button primary"]')):
        driver.find_element_by_xpath('//div[contains(@class,"ui_pagination")]//a[contains(@class,"nav next taLnk ui_button primary")]').click()
        time.sleep(5)
  return reviewList

In [0]:
#Creating a list of dictionaries containing all the reviews of all the restaurant from the RESTAURANT_LIST dataframe
url='https://www.tripadvisor.in/Restaurant_Review-g304551-d13388460-Reviews-Kitchen_With_A_Cause-New_Delhi_National_Capital_Territory_of_Delhi.html'
reviewList=get_review(url)


DATA CLEANING AND ANALYSIS

In [0]:
#Converting the review list into DataFrame
DF=pd.DataFrame(reviewList)

In [0]:
#Cleaning the review paragraph
clean_text(DF,4)

In [19]:
#Calculating the Most Frequent Words
def most_freq(count):
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize((" ".join(DF["Review Paragraph"])))
  word_count=Counter(tokens)
  return word_count.most_common(count)
print ("Top Tem Most Frequent Words: ")
for i,j in most_freq(10):
  print ("Word - %s : Occurance - %s"%(i,j))

Top Tem Most Frequent Words: 
Word - food : Occurance - 183
Word - great : Occurance - 100
Word - restaurant : Occurance - 100
Word - good : Occurance - 95
Word - place : Occurance - 71
Word - staff : Occurance - 63
Word - cause : Occurance - 57
Word - service : Occurance - 56
Word - visit : Occurance - 52
Word - delhi : Occurance - 51


In [0]:
#Copy for Data Analysis
nltk_DF=DF.copy()
clean_text(nltk_DF,2)
nltk_DF.iloc[:,4]=DF.iloc[:,2]+':'+DF.iloc[:,4]

In [0]:
#Calculation of Polarity Score
nltk_DF['POLARITY']=nltk_DF["Review Paragraph"].apply(lambda x: nltk_sent_cal(x)['compound'])

In [22]:
print ("MOST POSITIVE REVIEW")
pos_index=nltk_DF[nltk_DF.POLARITY==max(nltk_DF.POLARITY)].index[0]
print ("Review Polarity - %s"%nltk_DF["POLARITY"][pos_index])
print ("Review Title - %s"%DF["Review Title"][pos_index])
print ("Review Paragraph - %s"%DF["Review Paragraph"][pos_index])

MOST POSITIVE REVIEW
Review Polarity - 0.9955
Review Title - Such a beautiful experience and food, for such a great cause
Review Paragraph - mum go restaurant twice 2 separate stay delhi precisely love much find food reasonably price super tasty staff totally wonderful offer variety food choose love super trendy relax restaurant lovely atmosphere really like know ethos behind restaurant train teenager street child progress far super recommend not disappoint attach couple photo take place wonderful food justice


In [23]:
print ("MOST NEGATIVE REVIEW")
neg_index=nltk_DF[nltk_DF.POLARITY==min(nltk_DF.POLARITY)].index[0]
print ("Review Polarity - %s"%nltk_DF["POLARITY"][neg_index])
print ("Review Title - %s"%DF["Review Title"][neg_index])
print ("Review Paragraph - %s"%DF["Review Paragraph"][neg_index])

MOST NEGATIVE REVIEW
Review Polarity - -0.7011
Review Title - Kitchen without a cause
Review Paragraph - 5 star review restaurant give people cause concern like many westerner review restaurant also go dinner think place good food take age arrive overpriced not local standard much like tesco package food western palate use especially shahi paneer full sugar red colour no cashew paste traditional spice ghee grow locally know authentic food taste like not pay not eat take convince hakka noodle okay though paneer order give son stomach problem morning ambience poor bad acoustic timber palette hang ceil obviously designer restaurant see trendy western restaurant case specially make architect though layout poor family get drink make lot noise manager timid stop much worryingly not get proper answer charitable cause tell support street kid not see charity registration numb not see street kid 1 intern give interview facebook page give detail learn appear street kid good educate adult talk str

In [24]:
print ("NEUTRAL REVIEWS")
neu_index=nltk_DF[nltk_DF.POLARITY==0].index[0]
print ("Review Polarity - %s"%nltk_DF["POLARITY"][neu_index])
print ("Review Title - %s"%DF["Review Title"][neu_index])
print ("Review Paragraph - %s"%DF["Review Paragraph"][neu_index])

NEUTRAL REVIEWS
Review Polarity - 0.0
Review Title - Very tasty food
Review Paragraph - spend arround week india go meal time restaurant far one much tasty indian food come across staff professional polite


In [0]:
#Percentage of Positive and Negative Reviews
positive_percent=nltk_DF[nltk_DF.POLARITY>0].POLARITY.count()/nltk_DF.POLARITY.count()*100
negative_percent=nltk_DF[nltk_DF.POLARITY<0].POLARITY.count()/nltk_DF.POLARITY.count()*100

In [26]:
print ("Positive Reviews : %f"%positive_percent,'%')
print ("Negative Reviews : %f"%negative_percent,'%')

Positive Reviews : 97.727273 %
Negative Reviews : 1.704545 %


In [0]:
#Converting th review file into CSV
DF.to_csv('Data Analyst Internship Submission.csv',header=True,index=False)